# Run Molecular Dynamics Trajectories with Psi4 and TTM
We want to see how closely our ML models adhere to trajectories produced without surrogates

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from fff.sampling.md import MolecularDynamics
from fff.simulation.utils import read_from_string
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.calculators.psi4 import Psi4
from ase.io import write
from ttm.ase import TTMCalculator
from pathlib import Path
import pandas as pd
import numpy as np
import warnings

/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration

In [2]:
out_dir = Path('trajectories')
temp = 100

## Load in Example Structures
Use the same ones from our nwchem-evaluation

In [3]:
start_structs = pd.read_csv('../nwchem-evaluation/example_structures.csv')
print(f'Loaded {len(start_structs)} starting structures')

Loaded 20 starting structures


In [4]:
start_structs = start_structs.sort_values('ttm_energy', ascending=True).sort_values('n_waters')

## Run Molecular Dynamics with DFT
Run a 1000 timesteps and save an entry every 10

In [5]:
psi4 = Psi4(method='pbe0-d3', basis='aug-cc-pvdz', num_threads=12)

  Threads set to 12 by Python driver.


Test with diff

In [6]:
out_dir.mkdir(exist_ok=True)
for name, calc in [('ttm', TTMCalculator()), ('dft', psi4)]:
    for _, row in start_structs.iterrows():
        out_path = out_dir / f'{name}_n={row["n_waters"]}_T={temp}K-{row["id"]}.db'
        if out_path.is_file():
            continue

        # Read the structure and initialize it at 100K
        atoms = read_from_string(row['xyz'], 'xyz')
        MaxwellBoltzmannDistribution(atoms, temperature_K=100, rng=np.random.RandomState(1))

        # Run it for a few timesteps
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            _, traj = MolecularDynamics().run_sampling(atoms, 200, calc, timestep=0.1, log_interval=1)

        # Save it to disk
        write(str(out_path), traj)

  Threads set to 12 by Python driver.


KeyboardInterrupt: 